### 패션 아이템을 구분하는 DNN

**nn**
- 파이토치와 인공신경망 모델의 재료를 갖고 있는 모듈

**optim**
- 경사하강법을 이용하여 최적화 하기 위한 모듈

**Functional**
- 손실함수나 활성화 함수와 같이 계산을 하는 모듈

**torch.utils.data**
- 데이터셋의 표준 정의, 데이터셋을 불러오고, 자르고, 섞는 데 사용하는 도구들이 들어 있는 모듈

**data.DataLoader**
- 학습에 필요한 데이터를 로딩해 줌

**torchvision.datasets**
- 이미지 데이터셋의 모음

**torchvision.transforms**
- 데이터셋에 쓸 수 있는 여러가지 변환 필터를 담고 있는 모듈
- transforms.Compose()
  - 입력을 변환시키는 도구
- ToTensor()
  - 텐서로 변환
- Resize()
  - 이미지 크기를 조정
- Normalize()
  - 주어진 평균과 표준편차로 정규화
- RandomHorizontalFlip()
  - 무작위로 이미지의 오른쪽과 왼쪽을 뒤집는 기능
- RandomCrop()
  - 이미지를 무작위로 자르는 기능

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms

**torch.cuda.is_available**
- 현재 컴퓨터에서 CUDA를 이용할 수 있는지 알아보는 함수

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')
print(USE_CUDA)
print(DEVICE)

False
cpu


**훈련셋과 테스트셋 불러오기**
- 매개변수 train이 True이면 훈련셋, False이면 테스트셋

In [0]:
EPOCHS = 50
BATCH_SIZE = 64

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        './.data/',
        train=True,
        download=True,
        transform=transforms.Compose([
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307,),(0.3081),)
                  ])),
    batch_size=BATCH_SIZE,
    shuffle=True            
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        './.data/',
        train=False,
        download=True,
        transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307,),(0.3081),)
                  ])),
    batch_size=BATCH_SIZE,
    shuffle=True                   
)

**인공 신경망 구현**
- 레이어가 3개인 3층 인공신경망
- 입력: 784개의 특성
- 은닉층(fc1)의 유닛 수: 256개
- 은닉층(fc2)의 유닛 수: 128개
- 출력층(fc3): 10개의 패션 아이템 분류
- nn.Linear()
  - 선형 결합을 수행하는 객체를 생성
- forward()
  - 출력 예측값을 구하는 함수

In [0]:
class Net(nn.Module):
  def __init__(self, dropout_p=0.2):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(784, 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 10)
    self.dropout_p = dropout_p

  def forward(self, x):
    x = x.view(-1, 784)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training, p=self.dropout_p)
    x = F.relu(self.fc2(x))
    x = F.dropout(x, training=self.training, p=self.dropout_p)
    x = self.fc3(x)
    return x

**모델과 최적화 알고리즘 생성**

In [0]:
model = Net(dropout_p=0.2).to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

**모델 훈련 함수 생성**
- model.train()
  - 모델을 훈련 모드로 설정
- optimizer.zero_grad()
  - 반복할 때마다 기울기를 새로 계산하므로 초기화를 위한 함수
- output = model(data)
  - 훈련 데이터에 대한 모델의 예측값을 output 변수에 저장
- F.cross_entropy(output, target)
  - output과 target의 오차로 손실 계산
- loss.backward()
  - 기울기(가중치)의 변화율 계산
- optimizer.step()
  - 가중치 훈련

In [0]:
def train(model, train_loader, optimizer):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(DEVICE), target.to(DEVICE)
    optimizer.zero_grad()
    output = model(data)
    loss = F.cross_entropy(output, target)
    loss.backward()
    optimizer.step()

**성능 측정하기**
- reduction='sum'
  - 크로스 엔트로피에서 미니 배치의 평균 대신 합을 가져옴
- ouput.max()
  - 가장 큰 값과 위치값 반환
- pred.eq()
  - eq() 함수의 값이 일치하면 1, 아니면 0
- sum()
  - 모두 더하면 정답의 개수
- view_as()
  - target 텐서를 view_as() 함수 안에 들어가면 인수의 모양대로 다시 정렬함

In [0]:
def evaluate(model, test_loader):
  model.eval()
  test_loss = 0
  correct = 0

  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(DEVICE), target.to(DEVICE)
      output = model(data)
      test_loss += F.cross_entropy(output, target, reduction='sum').item()

      # 예측값과 정답을 비교하여 일치하는 경우 correct에 1을 더함
      pred = output.max(1, keepdim=True)[1]
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  test_accuracy = 100. * correct / len(test_loader.dataset)
  return test_loss, test_accuracy

In [0]:
for epoch in range(1, EPOCHS + 1):
  train(model, train_loader, optimizer)
  test_loss, test_accuracy = evaluate(model, test_loader)
  
  print('[{}] Test Loss: {:4f}, Accuracy: {:2f}%'.format(
      epoch, test_loss, test_accuracy))

### 이미지 처리 능력이 탁월한 CNN

In [0]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.conv2_drop = nn.Dropout2d()
    self.fc1 = nn.Linear(320, 50)
    self.fc2 = nn.Linear(50, 10)

  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
    x = x.view(-1, 320)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)
    return F.log_softmax(x, dim=1)

In [0]:
model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [0]:
for epoch in range(1, EPOCHS + 1):
  train(model, train_loader, optimizer)
  test_loss, test_accuracy = evaluate(model, test_loader)
  
  print('[{}] Test Loss: {:4f}, Accuracy: {:2f}%'.format(
      epoch, test_loss, test_accuracy))